Name(s):

# Association Analysis

Association analysis uses machine learning to extract frequent itemsets and strong association rules from large datasets. In this assignment you'll be implementing one of the most commonly used algorithms for association rule mining - the Apriori algorithm.

The dataset (`large_retail.txt`) that we are going to use has been adapted from the [Retail Market Basket Dataset](http://fimi.ua.ac.be/data/retail.pdf). This dataset contains transaction records supplied by a Belgian retail supermarket store. Each line in the file represents a separate transaction with the item ids separated by space. The dataset has 3000 transactions and 99 different item ids.

You are also provided with a smaller dataset (`small_retail.txt`) with 9 transactions and 5 different item ids along with the solutions. You can test and debug your implementation on this smaller dataset.

## Apriori Algorithm from scratch

The Apriori algorithm is a classical algorithm in data mining. It is used for mining frequent itemsets and relevant association rules. In this part, you'll be implementing this algorithm for generating the itemsets that occur enough times to meet the `min_sup` threshold.

**Implementation Hint:**

- Use the `frozenset` data structure in Python, which is similar to `set` in functionality, to represent the itemsets, because `frozenset` is an immutable (hashable) data structure. You can maintain a dictionary that maps from the itemset (a `frozenset`) to its support count.

In [775]:
# Standard imports (you can add additional headers if you wish)
import numpy as np
from itertools import combinations 

In [776]:
# Reading the dataset from file
def load_dataset(file_name):
    with open(file_name, 'r') as f:
        content = f.readlines()
        data = [[int(x) for x in line.rstrip().split()] for line in content]
    return data

In [777]:
# Read in the small_retail dataset
small_dataset = load_dataset('small_retail.txt')
small_dataset

large_dataset = load_dataset('large_retail.txt')
large_dataset

[[30, 31, 32, 39],
 [33, 34, 35, 32, 39],
 [36, 37, 38, 40, 42, 43, 44, 45, 46, 32, 39],
 [38, 39, 47, 48],
 [38, 39, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58],
 [41, 59, 60, 61, 62],
 [3, 39, 48],
 [63, 64, 65, 66, 67, 68, 32, 39],
 [69, 32, 39],
 [48, 70, 71, 72],
 [39, 73, 74, 75, 76, 77, 78, 79],
 [36, 38, 39, 41, 48, 79, 80, 81],
 [82, 83, 84, 32, 39],
 [41, 85, 86, 87, 88],
 [39, 48, 57, 63, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
 [36, 38, 39, 48, 89],
 [4, 7, 18, 28, 37, 42, 98, 32, 39],
 [2, 38, 95, 32, 39],
 [4, 15, 20, 30, 39, 56, 58, 98, 100],
 [3, 7, 22, 34, 38, 40, 41, 42, 43, 45, 51, 52, 53, 57, 74],
 [38, 48, 71, 93],
 [6, 9, 13, 29, 35, 38, 39, 42, 45, 48, 50, 58, 71, 78, 79],
 [1, 34, 39, 97],
 [18, 38, 39, 56, 87, 92],
 [42, 45, 48, 54, 71, 96],
 [39, 41, 48],
 [19, 47, 49, 59, 70, 73, 84, 32, 39],
 [19, 24, 38, 39, 48, 54, 62, 68, 92],
 [17, 20, 32, 39, 41, 48, 80, 95],
 [32, 38, 39, 47, 48, 67, 70, 91],
 [28, 39, 70, 85],
 [36, 38, 41, 48, 59, 69, 78],
 [1, 

Q1. Implement the function `create_1_itemsets` that takes as input the entire dataset and returns a list of all the candidate 1-itemsets. For example, for `small_retail.txt` it should return:
~~~
[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]
 ~~~
 Please don't hardcode the item ids, your code should support item ids that are non-sequential.

In [778]:


def create_1_itemsets(data):
    #base case: we have reached the end of our list
    candidates = {}
    transaction =0
    while(transaction != len(data)):
        for item in data[transaction]:
            candidates.update({item :frozenset({item})})
        transaction +=1

    return [frozenset({item}) for item in sorted(candidates.keys())]

can_list = create_1_itemsets(large_dataset)


In [779]:
#THIS IS BASED ON TRANSACTION REPITION/SIZE EFFECIENCY
#need a matrix (Transastions x items)
# 1 column for count and RC(duplicate transactions)

#input: a list of frozensets
def generate_index_map(candidates):
    item_idxs ={}
    i = 0
    while(i != len(candidates)):
        item_idxs[list(candidates[i])[0]] = i
        i+=1
    return item_idxs

def get_index(item, index_map):
    #print(item)
    return index_map[item]

#input: list
def convert_binary(candidate, allItems, index_map):
    byBinary = [0]*allItems
    for item in candidate:
        byBinary[get_index(item, index_map)] = 1
    return byBinary

    
def add_t_counts(bitmap):
      return np.concatenate((bitmap, bitmap.sum(axis=1, keepdims= True)), axis=1)
      

def remove_dupRows(bitmap):
    unq, cnt = np.unique(bitmap, axis =0, return_counts=True)
    return (unq, cnt)


def genMatrix(data, allItems, index_map):
    bitmap = []
    for transaction in data:
        #transaction = sorted(transaction)
        cur_row = convert_binary(transaction, allItems, index_map)
        bitmap.append(cur_row)
    bitmap = np.array(bitmap)
    bitmap2, dup_cnts = remove_dupRows(bitmap)
    #count of items in each transactions
    bitmap2 = add_t_counts(bitmap2)
    #count of duplicate transactions
    bitmap2 =np.hstack((bitmap2, [[cnt] for cnt in dup_cnts]))
    return bitmap2


Q2. Implement function `filter_candidates` that takes as input the candidate itemsets, the dataset, and the minumum support count `min_sup`, and filters out candidates that don't meet the support threshold.

Return a list of all the itemsets that meet `min_sup` (as a list of frozensets) and the support count information for all of them (as a `dict`).

In [780]:
#this function will return the minsup for a k-candidate 
#input: bitmap array, [0,1,1,0,0] candidate
def bitmap_supcount(bitmap, candidate):
    transactions = bitmap[:,0:len(candidate)]
    #generate mask based on frequent item set 
    mask = np.array([candidate]*len(transactions))
    #get all transaction where item x, item y exist
    masked_x = np.where(mask, transactions, 0)
    # need the index of all transactions bc need to multiple by how many 
    #accounts for transaction that have less than k items
    indices =np.all(mask == masked_x, axis=1).astype(int)
    # multiply the rows where the itemset exists by frequency of each transaction
    return np.sum(bitmap[:,-1] * indices)
    


#***********Thisis where we use the bitmap*************
def filter_candidates(bitmap, candidates, min_sup, index_map):
    retlist = []
    support_data = {}
    for candidate in candidates:
       
        #candidate = sorted(list(candidate))
        bybinary = convert_binary(candidate, len(bitmap[0])-2, index_map)
        sup_count = bitmap_supcount(bitmap, bybinary)
        if(sup_count >= min_sup):
            retlist.append(frozenset(candidate))
            support_data[frozenset(candidate)] = sup_count
    #retlist = sorted([frozenset(sorted(list(itemset))) for itemset in retlist], key = lambda x: len(x))
    #print(retlist)
    return retlist, support_data




In [790]:
del str

Q3. Implement the function `generate_next_itemsets` that takes in frequent itemsets of size `k` and generates candidate itemsets of size `k + 1`.

Use either the F(k-1) x F(k-1) or the F(k-1) x F(1) candidate generation method, then **filter the candidate list based on the apriori principle before returning it.**

In [781]:


#pass in freq_sets[first_kth_subset]
def check_subsets(candidate, k_sets,k): 
    candidate_subsets = sorted([list(x) for x in combinations(candidate, k)])
    #sorted(list(combinations(candidate, k)))
    for subset in candidate_subsets:
        cnt = 0
        subset = frozenset(subset)
        for kth_freq_set in k_sets:
            if kth_freq_set.issubset(subset):
                cnt += 1
                break
        if cnt == 0:
            return False
    return True

#get all the k_sets where they match in k-1 subsets          
def get_matches(k_sets, kth_idx, subset, k):
    next_k = kth_idx
    tojoin = []
    while(next_k != len(k_sets) and subset.issubset(k_sets[next_k])):
        tojoin.append(k_sets[next_k])
        next_k+=1
    candidates = join_itemsets(tojoin, k_sets, k)
    return next_k - kth_idx, candidates 

#join all the k sets
def join_itemsets(tojoin, k_sets, k):
    filter_candidates = []
    for i in range(len(tojoin)):
        for y in range(i + 1, len(tojoin)):
            k_candidate = frozenset(sorted(list(tojoin[i].union(tojoin[y]))))
            #print(k_candidate)
            if (check_subsets(k_candidate, k_sets, k)):
                filter_candidates.append(k_candidate)
    return filter_candidates

#input freq_sets with size k-1 and k
def generate_next_itemsets(subsets, k_sets, k):
    retlist =[]
    #base case: k = 1
    start_idx = 0
    if (subsets == []):
        candidates = [next(iter(oneset)) for oneset in k_sets]
        #removed list()
        candidates=  sorted([list(x) for x in combinations(candidates, 2)])
        return candidates
    else:
        #print(k_sets)
        for subset in subsets:
            k_joins, next_candidates = get_matches(k_sets, start_idx, subset, k)
            retlist += next_candidates
            start_idx += k_joins
            if(start_idx == len(k_sets)):
                return retlist
        

    return retlist
                

    

Q4. Implement the function `apriori_freq_itemsets` that takes the entire dataset as input and returns all the frequent itemsets that meet `min_sup`.

In [782]:
character_test =[ ['apples','bananas','d'], ['apples','d'], ['z'], ['g', 'e'], ['e'],]

def apriori_freq_itemsets(dataset, minsup):
    retlist = []
    support_data ={}
    one_can = create_1_itemsets(dataset)
    idx_map = generate_index_map(one_can)
    allItems = len(one_can)
    bitmap = genMatrix(dataset, allItems, idx_map)
    print(bitmap)
    #print(one_can)
    retlist, support_data = filter_candidates(bitmap, one_can, minsup, idx_map)
    k = 2
    s_subset_idx = 0
    e_subset_idx = 0
    start_idx =0
    while(len(retlist[-1]) == (k-1)):  
        itemsets = generate_next_itemsets(retlist[s_subset_idx:e_subset_idx], retlist[start_idx:], k-1)
        k_freq_set, sup_cnts = filter_candidates(bitmap, itemsets, minsup, idx_map)
        s_subset_idx = e_subset_idx
        e_subset_idx = retlist.index(retlist[-1]) + 1
        start_idx =  retlist.index(retlist[-1]) + 1
        retlist += k_freq_set
        support_data.update(sup_cnts)
        k+=1
    return retlist, support_data

retlist, support_data = apriori_freq_itemsets(large_dataset, 300)
retlist_small, support_data_small = apriori_freq_itemsets(small_dataset, 2)
#print(retlist)



[[ 0  0  0 ...  0  0 18]
 [ 0  0  0 ...  1  1  1]
 [ 0  0  0 ...  0  1  1]
 ...
 [ 1  0  0 ...  0  7  1]
 [ 1  0  0 ...  1 25  1]
 [ 1  1  0 ...  0 19  1]]
[[0 1 0 1 0 2 1]
 [0 1 1 0 0 2 2]
 [1 0 1 0 0 2 2]
 [1 1 0 0 1 3 1]
 [1 1 0 1 0 3 1]
 [1 1 1 0 0 3 1]
 [1 1 1 0 1 4 1]]


Q5. Display the frequent item sets in the form of a table along with their `support` (as a fraction: support count over number of transactions) for the `large_retail.txt` dataset **with a min support count of 300**.

Sample Table Format (tab separated table)

~~~
Sup     Freq Itemset
0.67	[1]
0.44	[1, 2]
(and so on)
...
...
~~~

`support(itemset) = support_count(itemset) / num_total_transactions`.

The `support` and the itemset should be separated by a tab (`'\t'`).

Note that the `support` should be rounded to the nearest 2 decimal places (use `round(sup, 2)`). If a support_fraction only contains 1 decimal place (for example, 0.1), you do not need to add a 0 to the end of it (leaving it as 0.1 is fine).

The itemsets should also be in a sorted order where smaller itemsets should come before larger itemsets and itemsets of the same size should be sorted amongst themselves.

For eg. 
~~~~
[1, 2] should come before [1, 2, 3]
[1, 2, 3] should come before [1, 2, 4]
[1, 2, 3] should come before [1, 4, 5]
[1, 2, 3] should come before [2, 3, 4]
~~~~

Note that **this order is very important for grading!** 

The output also shouldn't contain any duplicates. 

The sample output for the `small_retail.txt` dataset with `min_sup` set to 2 is:

~~~~
Sup     Freq Itemset
0.67	[1]
0.78	[2]
0.67	[3]
0.22	[4]
0.22	[5]
0.44	[1, 2]
0.44	[1, 3]
0.22	[1, 5]
0.44	[2, 3]
0.22	[2, 4]
0.22	[2, 5]
0.22	[1, 2, 3]
0.22	[1, 2, 5]
~~~~


In [791]:
# your code goes here
print('\t'.join(['Sup', 'Freq Itemset']))
for itemset in retlist:
    print('\t'.join([str(round(support_data[itemset]/len(large_dataset), 2)), str(sorted(list(itemset)))]))
print(" \nSMALL DATASET")
print('\t'.join(['Sup', 'Freq Itemset']))
for itemset in retlist_small:
    print('\t'.join([str(round(support_data_small[itemset]/len(small_dataset), 2)), str(list(itemset))]))


Sup	Freq Itemset
0.1	[31]
0.14	[32]
0.11	[36]
0.26	[38]
0.53	[39]
0.22	[41]
0.47	[48]
0.11	[60]
0.11	[65]
0.11	[89]
0.14	[32, 39]
0.15	[38, 39]
0.13	[38, 48]
0.14	[39, 41]
0.33	[39, 48]
0.18	[41, 48]
0.14	[39, 41, 48]
 
SMALL DATASET
Sup	Freq Itemset
0.67	[1]
0.78	[2]
0.67	[3]
0.22	[4]
0.22	[5]
0.44	[1, 2]
0.44	[1, 3]
0.22	[1, 5]
0.44	[2, 3]
0.22	[2, 4]
0.22	[2, 5]
0.22	[1, 2, 3]
0.22	[1, 2, 5]


Q6. Find the closed frequent item sets. Display results for the `large_retail.txt` dataset in the same format as specified in Q5.

The results for the `small_retail` dataset are as follows:

~~~~
Sup	Freq Itemset
0.0	[1]
0.0	[2]
0.0	[3]
0.0	[1, 2]
0.0	[1, 3]
0.0	[2, 3]
0.0	[2, 4]
0.0	[1, 2, 3]
0.0	[1, 2, 5]
~~~~

In [635]:
#I think this is correct????? -NEED TO CHECK*************************
def closed_freq_itemsets(support_data):
    closed_list =[]
    key_list = list(support_data.keys())
    endlist =  len(key_list)
    #check if this item is the max_sup subset of any other frequent itemset
    for i in range(endlist):
        curr_item = key_list[i]
        max_i_subset = key_list[i]
        for y in range(i, endlist):
            #print("curr max is", max_i_subset)
            other = key_list[y]
            #is this item a subset of support data?
            if(curr_item.issubset(other)):
                #does this the superset have the same support count?
                if(support_data[other] == support_data[curr_item]):
                    #print("replacing")
                    
                        max_i_subset = frozenset(sorted(list(other)))
        
        if max_i_subset not in closed_list:
              closed_list.append(max_i_subset)

    return sorted([sorted(list(itemset)) for itemset in closed_list], key = lambda x: len(x))


print('\t'.join(['Sup', 'Freq Itemset']))
output = closed_freq_itemsets(support_data)
for itemset in output:
    print('\t'.join([str(round(support_data[frozenset(itemset)]/len(large_dataset), 2)), str(list(itemset))]))

Sup	Freq Itemset
0.1	[31]
0.11	[36]
0.26	[38]
0.53	[39]
0.22	[41]
0.47	[48]
0.11	[60]
0.11	[65]
0.11	[89]
0.14	[32, 39]
0.15	[38, 39]
0.13	[38, 48]
0.33	[39, 48]
0.18	[41, 48]
0.14	[39, 41, 48]


## Extra Credit (+5 points)

Q7. Now, generate the strong rules from the frequent itemsets. Given a dictionary of frequent itemsets and their supports, generate all the rules having confidence above some provided `min_conf` threshold. Display the rules in the form of a table.

Sample table format (tab separated table):

~~~
Sup     Conf    Rule
0.44	0.67	[1] -> [2]
0.22	1.0	 [5] -> [1, 2]
0.22	1.0	 [2, 5] -> [1]
(and so on)
...
...
~~~

Rule confidence should be rounded to the nearest 2 decimal places (use `round(conf, 2)`). This table should also be tab (`'\t'`) separated. The rules should be displayed in a sorted order. If a rule is given as `LHS -> RHS` then the rules for which `len(LHS)` is lesser should appear first. If the `len(LHS)` is equal for two rules then rules for which `len(RHS)` is lesser should appear first. If both `len(LHS)` and `len(RHS)` is equal then the rules should be sorted based on LHS first and then based on RHS.

~~~~
Note:
LHS (Left Hand Side)
RHS (Right Hand Side)
~~~~

For eg.
~~~~
[3] -> [2] should come before [1, 3] -> [4]
[4] -> [2] should come before [2] -> [3, 4]
[1, 3] -> [2] should come before [1, 5] -> [2]
[1, 2] -> [3] should come before [1, 2] -> [5]
~~~~

Note that **this order is important for grading**. 

The sample output for the `small_retail` dataset with `min_conf = 0.5` is:

~~~~
Sup	 Conf	Rule
0.44	0.67	[1] -> [2]
0.44	0.67	[1] -> [3]
0.44	0.57	[2] -> [1]
0.44	0.57	[2] -> [3]
0.44	0.67	[3] -> [1]
0.44	0.67	[3] -> [2]
0.22	1.0	 [4] -> [2]
0.22	1.0	 [5] -> [1]
0.22	1.0	 [5] -> [2]
0.22	1.0	 [5] -> [1, 2]
0.22	0.5	 [1, 2] -> [3]
0.22	0.5	 [1, 2] -> [5]
0.22	0.5	 [1, 3] -> [2]
0.22	1.0	 [1, 5] -> [2]
0.22	0.5	 [2, 3] -> [1]
0.22	1.0	 [2, 5] -> [1]
~~~~

In [846]:

def pruned_check(con, pruned_dict):
    k = 0
    flag = True
    while (k >= len(con)):
        subsets = combinations(con, k)
        for subset in subsets:
            if subset in pruned_dict:
                #invalid
                if pruned_dict[frozenset(subset)] == False:
                    return False
            else:
                #set and keep valid for now
                pruned_dict[frozenset(subset)] == True
        k +=1
    return True
    
            
def generate_rules(support_data, minconf):
    conf_list =[]
    for item in support_data.keys():
        pruned_dict = {}
        right_size = 1
        left_size = len(item) - right_size
        sup_cnt =support_data[item]
        itemset = sorted(list(item))
        while(left_size != 0):
            con_list = list(combinations(itemset, right_size))
            for con in con_list:
                ant = frozenset(itemset).difference(frozenset(con))
                if(pruned_check(con, pruned_dict)):
                    ant_list = list(combinations(ant, left_size))
                    for a in ant_list:
                        conf = round(sup_cnt/ support_data[frozenset(a)], 2)
                        if conf >= minconf:
                            conf_list.append([round(sup_cnt/len(small_dataset),2), list(a), list(con), conf])
                        else:
                            pruned_dict[frozenset(con)] = False
            left_size -=1
    conf_list = sorted(conf_list, key = lambda x: (len(x[1]), x[1]))
    return conf_list
        


output2 = generate_rules(support_data_small, .5)
#print(output2)


# your code goes here
#got every dict in the list
print("SMALL RETAIL")
print('\t'.join(['Sup', 'Conf', 'Freq Itemset']))
for i in output2:
    toret = str(i[0]) + "\t" +  str(i[3])  + "\t" + str(i[1]) +" " + ' -> ' + " "+ str(i[2])
    print(toret)

output2 = generate_rules(support_data, .5)
print("\n LARGE RETAIL")
print('\t'.join(['Sup', 'Conf', 'Freq Itemset']))
for i in output2:
    toret = str(i[0]) + "\t" +  str(i[3])  + "\t" + str(i[1]) +" " + ' -> ' + " "+ str(i[2])
    print(toret)



    
    # your code goes here

SMALL RETAIL
Sup	Conf	Freq Itemset
0.44	0.67	[1]  ->  [2]
0.44	0.67	[1]  ->  [3]
0.44	0.57	[2]  ->  [1]
0.44	0.57	[2]  ->  [3]
0.44	0.67	[3]  ->  [1]
0.44	0.67	[3]  ->  [2]
0.22	1.0	[4]  ->  [2]
0.22	1.0	[5]  ->  [1]
0.22	1.0	[5]  ->  [2]
0.22	1.0	[5]  ->  [1]
0.22	1.0	[5]  ->  [2]
0.22	0.5	[1, 2]  ->  [3]
0.22	0.5	[1, 2]  ->  [5]
0.22	0.5	[1, 3]  ->  [2]
0.22	1.0	[1, 5]  ->  [2]
0.22	0.5	[2, 3]  ->  [1]
0.22	1.0	[2, 5]  ->  [1]

 LARGE RETAIL
Sup	Conf	Freq Itemset
46.67	1.0	[32]  ->  [39]
49.89	0.58	[38]  ->  [39]
42.67	0.5	[38]  ->  [48]
109.11	0.62	[39]  ->  [48]
47.56	0.64	[41]  ->  [39]
59.33	0.8	[41]  ->  [48]
47.56	0.64	[41]  ->  [39]
47.56	0.64	[41]  ->  [48]
109.11	0.7	[48]  ->  [39]
47.56	1.0	[41, 39]  ->  [48]
47.56	0.8	[48, 41]  ->  [39]


## Part 2 - Association Analysis using MLXtend

## Installation
`scikit-learn` does not provide any functionality for association rule mining so for this assignment you will be using the `MLxtend` library. The documentation for this library is available [here](http://rasbt.github.io/mlxtend/).

You will need to install the `MLxtend` library. There are several ways of doing this; you can follow the instructions below, or see the setup guide  [here](http://rasbt.github.io/mlxtend/installation/).


### Conda

If you downladed Anaconda in order to get Jupyter Notebooks (which is the most common way to get Jupyter Notebooks), then you will want to install `MLxtend` using Conda. 

Open a command prompt / terminal window and type:

`conda install mlxtend --channel conda-forge`

### PyPi

You can also install via pip. 
Note: If you are running Jupyter notebooks through an Anaconda install, then pip may not place the `MLxtend` libraries in the correct place for use in Jupyter (which is why you should use Conda instead of pip, if you downloaded Anaconda).

Open a command prompt / terminal window and type:

`pip3 install mlxtend`

### Google Colab

If you are using Google Colab, you can install MLXtend via a pip install command with an exclamation point in front of it, in one of your notebook cells. The exclamation points allows you to put shell commands inside of Colab. 

In a cell in your Colab notebook, type:

`!pip install mlxtend`


## Dataset
For the second part of this assignment, the data we'll use comes from a bakery called "The Bread Basket", located in the historic center of Edinburgh. The dataset contains more than 9000 transactions from the bakery. The file contains the following columns:

- Date. Categorical variable that tells us the date of the transactions (YYYY-MM-DD format). The column includes dates from 2016-10-30 to 2017-04-09.

- Time. Categorical variable that tells us the time of the transactions (HH:MM:SS format).

- Transaction. Quantitative variable that allows us to differentiate the transactions. The rows that share the same value in this field belong to the same transaction.

- Item. Categorical variable with the products purchased.

In this part, you'll be running the Apriori algorithm from the MLxtend library to generate the itemsets that occur more than the `min_sup` threshold. Based on these frequent itemsets, you'll find association rules that have confidence above the `min_conf` threshold.

In [ ]:
# Standard imports (you can add additional headers if you wish)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpmax
from mlxtend.frequent_patterns import association_rules

In [ ]:
%matplotlib inline

In [ ]:
# Reading the dataset from file
def load_dataset(file_name):
    with open(file_name, 'r') as f:
        content = f.readlines()[1:]
    transactions = []
    prev_tid = -1
    for t in content:
        t = t.strip().split(',')[-2:]
        tid = t[0]
        item = t[1]
        if prev_tid != tid:
            prev_tid = tid
            transactions.append([item])
        else:
            transactions[-1].append(item)
    return transactions

In [ ]:
dataset = load_dataset('BreadBasket_DMS.csv')

# ** NOTE: dataset is a 2D list (not a dataframe!) **

print("Num transactions:", len(dataset))
#Print the first 10 transactions
dataset[:10]

Q8. Data Cleaning: Many transactions in the dataset include the item "NONE." First, find and remove all the "NONE" items from the dataset. There are some transactions that only contain "NONE," so removing "NONE" will leave some transactions as empty lists. Remove all the empty lists as well. 

Once you have removed the NONEs, find the top 10 best-selling items in the bakery. Create a bar chart to display the total number of transactions for each of the top 10 selling items. Sort the bar chart by frequency (the top most sold item first, down to the 10th most sold item).

In [ ]:
for list in dataset:
    if "NONE" in list:
        list.remove("NONE")
for list in dataset:
    if not list:
        dataset.remove(list)
freq = {}
for list in dataset:
    for item in list:
        if freq.get(item) != None:
            freq[item] = freq[item] + 1
        else:
            freq[item] = 1
i = 0
for item in sorted(freq, key=freq.get, reverse=True):
    print(item, freq[item])
    i+=1
    if i == 10:
        break


Q9. 

a.) Using `mlxtend.preprocessing.TransactionEncoder`, transform `dataset` into an array format suitable for the `mlxtend` library. You will need to call `fit` then `transform`. 

`TransactionEncoder` learns unique items from the dataset and transforms each transaction into a one-hot encoded boolean numpy array. For example, the resulting encoded dataset will be represented by something like this, where each row is a transaction. If the first transaction contained ['Crepe', 'Jam'], this would correspond to the first row in the encoded table. 

<img src="table.png">

Print the `shape` of the resulting encoded numpy array.

b.) `TransactionEncoder` also has a function `inverse_transform` that allows you to tranform one-hot encoded transactions back to the item labels. Try it out on the first 5 transactions and display the items in the first 5 transactions.

In [ ]:
# your code goes here

Q10. Convert the encoded numpy array from Q4 part a into a pandas dataframe. Use the `TransactionEncoder`'s `.columns_` attribute as the column headers. Print the head of the resulting dataframe. 

In [ ]:
# your code goes here

Q11. Use the `mlxtend.frequent_patterns.apriori` to generate the frequent itemsets with minimum support of 1% (0.01). Display these itemsets along with their support values.

In [ ]:
# your code goes here

Q12. Use `mlxtend.frequent_patterns.fpmax` to find and display all of the maximal frequent itemsets along with their support values. 

In [ ]:
# your code goes here

Q13. Use `mlxtend.frequent_patterns.association_rules` to calculate rules with a confidence level of 0.25 for the frequent itemsets you generated in Q11.

In [ ]:
# your code goes here

Q14. An important step in generating a set of association rules is to determine the optimal thresholds for support and confidence. If we set these values too low we will get a lot of rules and most of them will not be useful. 

Generate the frequent itemsets with minimum support of 0.5% and plot the number of rules generated with respect to the confidence threshold by varying min_conf between 0 and 1 with increments of 0.1. Notice what happens when you increase the confidence level.

In [ ]:
# your code goes here

Q15. What value would you choose for the minimum confidence threshold based on the previous plot? Explain why as a comment. 

Display the rules generated for the your chosen value. Take a look at the generated rules. Are they interesting? As a comment, explain why or why not.

In [ ]:
# your code goes here